 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 1: Outils pour l'algèbre linéaire et l'optimisation
Tangi Migot

In [ ]:
using Pkg; 
Pkg.activate("jump")

## JuMP: un langage de modélisation

### JuMP ?
JuMP est le language de modélisation pour l'optimisation natif dans Julia. En d'autres termes, on peut modéliser des problèmes d'optimisation en Julia grâce à ce package.

JuMP fait partie de l'organisation [jump-dev](https://jump.dev/) qui propose un ensemble d'outils concernant l'optimisation.

On pourra trouver beaucoup d'informations et d'exemples dans la documentation de JuMP [jump.dev/JuMP.jl/stable/](https://jump.dev/JuMP.jl/stable/)

### Installer JuMP
Comme on l'a fait pour `LinearAlgebra` il faut installer le package `JuMP` via le *package manager*.

In [ ]:
] add JuMP

Ensuite, on peut ajouter `JuMP` a notre environement et commencer à l'utiliser.

In [ ]:
using JuMP

### Installer un solveur
Dans cet exemple, on va ajouter [`Ipopt`](https://en.wikipedia.org/wiki/IPOPT) qui est un package qui fournit un solveur pour l'optimisation nonlinéaire.

In [ ]:
] add Ipopt

In [ ]:
#ou alors:
#using Pkg
#Pkg.add("Ipopt")

In [ ]:
using Ipopt

In [ ]:
? ipopt

On installe aussi `MathOptInterface` qui permet de communiquer entre le modèle et le solveur.

In [ ]:
using MathOptInterface

### Structure d'un modèle JuMP
Tous les modèles écrits avec JuMP ont la même structure:
- un objet `Model`, c'est l'initialisation de notre modèle.
- un objet `Optimizer`, qui spécifie un solveur.
- `Variables`, les variables du problème
- `Objective`, la fonction objectif
- `Constraints`, les contraintes.

#### Le modèle
Tous les problèmes modéliser avec JuMP sont des objets `Model`. Cet objet sera ensuite toujours mentionné lorsque l'on ajoutera des variables, la fonction objectif et des contraintes.

In [ ]:
MonModel = Model()

Il est également possible d'associer un solveur à ce modèle.

In [ ]:
MonModel = Model(Ipopt.Optimizer) #utilise Ipopt comme solveur

#### Les variables
Une première donnée importante de notre problème d'optimisation est la variable.

In [ ]:
@variable(MonModel, 0 <= x <= 10)

In [ ]:
unregister(MonModel, :x) # if you want to remove the variable x from the model

A ce stade, on peut aussi spécifier des contraintes de bornes sur les variables.

In [ ]:
n = 10
l = [1; 2; 3; 4; 5; 6; 7; 8; 9; 10]
u = [10; 11; 12; 13; 14; 15; 16; 17; 18; 19]
@variable(MonModel, l[i] <= x[i=1:n] <= u[i])

ou plus simplement

In [ ]:
@variable(MonModel, xx[i=1:n] >= 0)

#### La fonction objectif
La fonction objectif se définit à l'aide de `@objective` et a trois arguments: le nom du modèle, le type d'optimisation (min ou max), et la fonction.

In [ ]:
c = [2; 3; 4; 5] 
@objective(MonModel, Min, sum(c[i]*x[i] for i in 1:4))

On peut aussi l'écrire sous forme de problème de maximisation.

In [ ]:
@objective(MonModel, Max, sum(c[i]*x[i] for i in 1:4))

Si la fonction objectif est nonlinéaire, on peut être plus précis et utiliser `@NLobjective`.

In [ ]:
@NLobjective(MonModel, Min, exp(x[1]))

#### Les contraintes

In [ ]:
TonModel = Model()
@variable(TonModel, x) #on peut avoir plusieurs types de variables
@variable(TonModel, y)
@constraint(TonModel, 5*x + 3*y <= 5) #une première contrainte

Il peut être utile de nommer une contrainte pour s'y réferer ensuite.

In [ ]:
@constraint(TonModel, un_nom_de_contrainte, 6*x + 4*y >= 5)

On peut aussi ajouter plusieurs contraints simultanément avec @constraints

In [ ]:
@constraints(TonModel, begin
           2x <=  1
            x >= -1
       end)

In [ ]:
TonModel[:un_nom_de_contrainte]

Si la contrainte est nonlinéaire, on peut être plus précis et utiliser `@NLconstraint`.

In [ ]:
@NLconstraint(TonModel, x^2 == 0)

On peut également introduire des contraintes plus génériques à l'aide de fonctions.

In [ ]:
a = [1; -3; 5; 7] 
@variable(TonModel, w[1:4])
@constraint(TonModel, sum(a[i]*w[i] for i in 1:4) <= 3)

On peut utiliser une boucle pour définir plusieurs contraintes.

In [ ]:
@constraint(TonModel, conRef3[i in 1:3], 6*x + 4*y >= 5*i)
#= ou identiquement:
for i in 1:3
    @constraint(yourModel, 6*x + 4*y >= 5*i)
end
=#

#### Exemple d'un code complet: Optimisation linéaire
On résout ici un problème d'optimisation linéaire
$$
\min_{x \in R^n} - x - 2y \ \text{sujet à} \ x + y \leq 1, 0 \leq x,y \leq 1.
$$

In [ ]:
model = Model(Ipopt.Optimizer)

On définit les variables.

In [ ]:
@variable(model, 0 <= x <= 1)
@variable(model, 0 <= y <= 1)

La contrainte $x + y \leq 1$.

In [ ]:
@constraint(model, x + y <= 1)

Enfin, on ajoute la fonction objectif.

In [ ]:
@objective(model, Min, -x - 2y)

Pour résoudre ce problème, on peut utiliser la fonction `optimize`.

In [ ]:
JuMP.optimize!(model)

Une fois la résolution terminée, on peut vérifier le status de la résolution.

In [ ]:
@show JuMP.has_values(model)
@show JuMP.termination_status(model) == MathOptInterface.LOCALLY_SOLVED
@show JuMP.primal_status(model) == MathOptInterface.FEASIBLE_POINT
@show JuMP.dual_status(model) == MathOptInterface.FEASIBLE_POINT

On peut également faire une analyse sur la solution obtenue.

In [ ]:
@show JuMP.value(x)              # Old syntax: getvalue(x)
@show JuMP.value(y)              # Old syntax: getvalue(y)
@show JuMP.objective_value(model)       # Old syntax: getobjectivevalue(model)

#### Exemple d'un code complet: Optimisation nonlinéaire
$$
\min_{x,y} exp(x)+y \text{ s.à } exp(x) + sin(x) <= 0
$$

In [ ]:
model = Model(Ipopt.Optimizer)
@variable(model, x)
@variable(model, y)

@NLobjective(model, Min, exp(x)+y)
@NLconstraint(model, exp(x)+sin(x) <=0)

In [ ]:
optimize!(model)
println("x = ", value(x), " y = ", value(y))

In [ ]:
termination_status(model)

In [ ]:
vals = zeros(2)
x_index = JuMP.index(x)
y_index = JuMP.index(y)

In [ ]:
d = NLPEvaluator(model)

In [ ]:
MathOptInterface.initialize(d, [:Grad])

In [ ]:
MathOptInterface.eval_objective(d, zeros(2))

In [ ]:
∇f = zeros(2)
MathOptInterface.eval_objective_gradient(d, ∇f, zeros(2))
∇f

In [ ]:
J = zeros(1,2)
MathOptInterface.eval_constraint_jacobian(d, J, zeros(2))
J

#### Plus d'information
Dans ce cours, nous n'allons pas uniquement nous concentrer sur des problèmes linéaires, mais aussi nonlinéaire.
Vous trouverez plus d'informations pour ce cas dans la documentation de jump-dev:

[https://jump.dev/JuMP.jl/stable/tutorials/nonlinear/introduction/](https://jump.dev/JuMP.jl/stable/tutorials/nonlinear/introduction/)

et la documentation de MathOptInterface:

[https://jump.dev/MathOptInterface.jl/stable/](https://jump.dev/MathOptInterface.jl/stable/)

Et ça sera le sujet du projet !